# POSITIONAL INDEXES LOADING

In [8]:
from termcolor import colored
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm

wremove = ['-','â€','â€¦','”']
filedata = []
def remove_prepositions(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    filtered = [word for word, pos in tagged if pos not in ['IN', 'TO']]
    return ' '.join(filtered)
# CREATING INSTANCES OF STEMMING AND LEMMITIZATION
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
translator = str.maketrans('', '', string.punctuation.replace('-', ''))
documents = []
wholecontent = ""
# READING ALL 30 FILES AND APPLY PREPROCESSING 
for x in range(1, 31):
    t = "C:\\Users\\DELL\\Desktop\\boolean\\{}.txt".format(x)
    with open(t, "r") as f:
        contents = f.read()
        # FOR REMOVING LINKS AND SPECIAL CHARACTRS FROM FILE
        contents = re.sub(r"&nbsp;.*\n", " ", contents)
        contents = re.sub(r"â€¢.*\n", " ", contents)
        # STEMMING PROCESS
        contents = nltk.word_tokenize(contents)
        contents = [stemmer.stem(word) for word in contents]
        contents = ' '.join(contents)
        # LEMMITIZATION PROCESS
        contents = " ".join([lemmatizer.lemmatize(word, pos='v') if pos.startswith('V') else lemmatizer.lemmatize(word) for word, pos in nltk.pos_tag(nltk.word_tokenize(contents))])
        # REMOVING PUNCTUATION MARKS
        contents = contents.translate(translator)
        # APPLYING CASE FOLDING
        contents = contents.lower()
        # GATHERING ALL PREPROCESSED DATA
        filedata.append(contents)
        wholecontent += contents
        contents = ""
# BUILDING INDEXES
wholecontent = wholecontent.split()
wholecontent = set(wholecontent)
wholecontent = sorted(list(wholecontent))
wholecontent = [element for element in wholecontent if element not in wremove]

positional_index = {}
# LOADING BAR PARALLEL WITH INDEX BUILDING
for term in tqdm(wholecontent, leave=False):
    positional_index[term] = {}
    for x in range(1, 31):
        data = filedata[x-1].split()
        if term in data:
            # INSERTING POSITION OF TERMS IN EACH DOCUMENT
            positions = [pos for pos, word in enumerate(data) if word == term]
            positional_index[term][x] = positions




# YOUR DICTIONARY

In [9]:
for key, value in positional_index.items():
    print(colored(key, "red", attrs=["bold", "underline"]), ":", colored(f"[{value}]", "green", attrs=["bold"]), "\n")

0 : [{9: [35], 16: [27], 23: [33, 174]}] 

04-feb-2023 : [{23: [28]}] 

1 : [{1: [1470], 6: [600], 10: [293], 16: [482], 17: [108], 27: [346]}] 

1-0 : [{22: [376]}] 

1-1 : [{30: [77]}] 

1-13 : [{30: [30]}] 

1-16 : [{4: [77]}] 

1-4 : [{30: [42]}] 

10 : [{9: [71, 535], 18: [735], 22: [438], 24: [198]}] 

10-over : [{28: [674]}] 

100 : [{12: [228], 26: [318, 386]}] 

100-run : [{23: [469]}] 

1002nd : [{7: [573]}] 

101 : [{30: [24]}] 

103 : [{15: [333], 27: [768]}] 

10th : [{9: [430], 18: [679]}] 

11 : [{7: [20, 131], 13: [221], 17: [166], 28: [236], 30: [645]}] 

111 : [{2: [751]}] 

112 : [{16: [35, 158], 23: [31, 172]}] 

113 : [{16: [25]}] 

115 : [{15: [40, 156]}] 

116 : [{23: [438]}] 

118 : [{28: [35]}] 

11am : [{5: [609]}] 

11th : [{15: [292], 22: [1607], 30: [738]}] 

12 : [{1: [415], 2: [246], 7: [1146], 18: [184], 22: [1472], 23: [428], 25: [430], 30: [790]}] 

120 : [{19: [32]}] 

122 : [{18: [270]}] 

1225 : [{22: [106]}] 

125 : [{27: [526]}] 

126 : [{27: [29,

# APPLY PROXIMITY QUERIES

In [10]:
import re
import ipywidgets as widgets
from IPython.display import display

# defining regular expression for validating queries

pattern = r'^[a-zA-Z]+\s[a-zA-Z]+\s/\s[1-9][0-9]*$'

# Define the input widgets.
query_input = widgets.Text(placeholder='Enter a boolean query')

# Define the output widgets.
output_label = widgets.Label()
output_list = widgets.SelectMultiple()

# Define the button widget and its click event handler.
button = widgets.Button(description='Search')
def on_button_click(_):
    query = query_input.value
    match = re.match(pattern, query)
    if match:
    # MAKING ARRAY OF QUERY ELEMENTS
        query_terms = query.split()
        proximity = query_terms[3]
        proximity = int(proximity)
        # Applying lemmitization and stemming to query terms
        for x in range(0, 2):
            query_terms[x] = query_terms[x].lower()
            query_terms[x] = stemmer.stem(query_terms[x])
            query_terms[x] = lemmatizer.lemmatize(query_terms[x])
        result = set()
        # Processing proximity query
        if query_terms[0] in positional_index and query_terms[1] in positional_index:
            docs1 = positional_index.get(query_terms[0], {})
            docs2 = positional_index.get(query_terms[1], {})
            for doc_id, positions in docs1.items():
                if doc_id in docs2:
                    other_positions = docs2[doc_id]
                    for pos1 in positions:
                        for pos2 in other_positions:
                            if abs(pos1-pos2) <= proximity:
                                result.add(doc_id)
                                break
            if len(result)>0:
                output_label.value = 'Results:'
                output_list.options = result
            else:
                output_label.value = 'Not found'
        else:
            output_label.value = 'Not found'
    else:
        output_label.value = 'Invalid Query'
button.on_click(on_button_click)

# Create a container to hold the widgets.
input_box = widgets.VBox([query_input, button])
output_box = widgets.VBox([output_label, output_list])
container = widgets.HBox([input_box, output_box])

# Display the container.
display(container)
